In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.metrics import accuracy_score

%pylab inline

### Load Train Data

In [ ]:
# Load Train dataset
train_data = pd.read_csv('mnist_train.csv')
train_data.shape

### Load Test Data

In [ ]:
# Load Test dataset
test_data = pd.read_csv('mnist_test.csv')
test_data.shape

### Separate Labels

In [ ]:
# Separate train labels
train_labels = train_data.label

del train_data['label']

# Transform train labels to classes for cross_entropy
train_labels = pd.get_dummies(train_labels)

# Separate test labels
test_labels = test_data.label

del test_data['label']

### Reshape to (x, 1, 28, 28)

In [ ]:
# Reshape train data
train_data = train_data.values.reshape((60000, 28, 28, 1))
print(train_data.shape)

# Reshape test data
test_data = test_data.values.reshape((10000, 28, 28, 1))
print(test_data.shape)

### Visualize Some Digits

In [ ]:
# Which item to show
ITEM_TO_SHOW = 33

# The first column is the label
label = train_labels.iloc[ITEM_TO_SHOW, :].idxmax()

# The rest of columns are pixels
pixels = train_data[ITEM_TO_SHOW, :, :, 0]

# Make those columns into a array of 8-bits pixels
# This array will be of 1D with length 784
# The pixel intensity values are integers from 0 to 255
pixels = np.array(pixels, dtype='uint8')

# Plot
plt.title('Label is {label}'.format(label=label))
plt.imshow(pixels, cmap='gray')
plt.show()

### Scale Data

In [ ]:
# Scale data to 0..1
train_data = train_data / 255.
test_data = test_data / 255.

### Model Creation

In [ ]:
kristofari = Sequential()

# Layer 1
kristofari.add(Conv2D(32, kernel_size=(3, 3), data_format = 'channels_last', input_shape=(28, 28, 1)))
kristofari.add(Activation('relu'))

# Layer 2
kristofari.add(Conv2D(64, (3, 3)))
kristofari.add(Activation('relu'))

# Layer 3
kristofari.add(MaxPooling2D(pool_size=(2, 2)))
kristofari.add(Dropout(0.25))               

# Layer 4
kristofari.add(Flatten())
kristofari.add(Dense(128))
kristofari.add(Activation('relu'))
kristofari.add(Dropout(0.5))               

# Layer 5
kristofari.add(Dense(10))
kristofari.add(Activation('softmax'))

kristofari.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
kristofari.summary()

### Model Training

In [ ]:
%%time

# Required constants
MINIBATCH_SIZE = 64
NUM_OF_EPOCHS = 10

history = kristofari.fit(train_data, train_labels, batch_size=MINIBATCH_SIZE, epochs=NUM_OF_EPOCHS, verbose=1)

### Let's Predict!

In [ ]:
# Here our neural network makes prediction on _unseen_ test data
prediction = kristofari.predict(test_data)

# Transform a little bit
prediction = np.apply_along_axis(argmax, 1, prediction)

# Finally print the accuracy
final_score = accuracy_score(test_labels, prediction)
print(f"Correctly predicted in {final_score:.2%} cases out of {test_labels.shape[0]} samples!")

### Now Check AI on Own Digits :)

In [ ]:
# Read own own hand-written digit and show it
img = cv2.imread("./Peter_Digit_07.png", 0)
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
# Resize to be 28x28
img = cv2.resize(img, (28, 28))

# Scale to 0..1
img = img / 255.

# What digit might be there?
prediction = kristofari.predict(img.reshape((1, 28, 28, 1)))
print(f"Kristofari thinks that it's {prediction.argmax()}")